In [ ]:
import polars as pl
import fastparquet as fp
import pandas as pd
import json
import os
import pyarrow.parquet as pq

In [ ]:
with open('generated.json', 'r') as f:
    data_json = json.load(f)
    
len_json = len(data_json)

print("length of json: ",len_json)

if "doc_info.parquet" not in os.listdir("."):
    len_doc = 0
else:
    len_doc = pq.read_metadata("doc_info.parquet").num_rows
    
print("length of docinfo: ",len_doc)

try:
    for i,keyword_obj in enumerate(data_json):
        
        key_loc = keyword_obj['doc_loc']
        key_arr = keyword_obj['keywords_array']

        for keyarr_obj in range(len(key_arr)):
            
            key_obj = key_arr[keyarr_obj]
            keyword_name = key_obj['keyword']
            keyword_score = key_obj['score']
            print(i,keyword_name,keyword_score)

            if keyword_name + "Index.parquet" not in os.listdir("."):
                
                dict = {"id" : len_doc, "score":keyword_score}
                df = pl.from_dict(data = dict, schema={"id" : pl.Int32, "score" : pl.Float64})
                df = df.to_pandas()
                df.to_parquet("./" + keyword_name + "Index.parquet")

        if "doc_info.parquet" not in os.listdir("."):
            dict = {"id" : [],"location":[]}
            df = pl.from_dict(data = dict, schema={"id" : pl.Int32, "location" : pl.Utf8})
            df = df.to_pandas()
            df.to_parquet("./doc_info.parquet")
            
        data = {"id" : [len_doc + i],"location": [key_loc] }
        df1 = pd.DataFrame(data)
        fp.write("doc_info.parquet",df1,append = True)
        
except:
    print("Some error occured")

mapping docid docloc

In [3]:
import numpy as np

def append_data(json_data,dataset):

    len_dset = len(dataset)

    for i,val in enumerate(json_data):
        data = np.array([(len_dset + i , val['doc_loc'])], dtype=[('doc_id', 'int32'), ('doc_loc', 'S10')])
        dataset.resize(dataset.shape[0]+1, axis=0)   
        dataset[len_dset + i] = data

In [4]:
import h5py
import json
import numpy as np

with open('generated.json', 'r') as f:
    json_data = json.load(f)

len_json_data = len(json_data)
print("len_json_data",len_json_data)

file = h5py.File('doc_info1.hdf5', 'a')

if 'my_dataset' not in file:
    dataset = file.create_dataset('my_dataset', shape=(0,), dtype=[('doc_id', 'int32'), ('doc_loc', 'S10')],maxshape=(None,))
else:
    dataset = file['my_dataset']

len_json_data 2


In [5]:
append_data(json_data,dataset)

print("len dset: ",len(dataset))

for i,val in enumerate(dataset):
    print(i,val)

len dset:  2
0 (0, b'640c08980f')
1 (1, b'640c0898a3')


generating keywords file

In [2]:
import vaex
import json 
import h5py

# df = vaex.open('doc_info1.hdf5')
df = h5py.File('doc_info1.hdf5', 'a')
# for i,val in enumerate(df.items()):
#     print(i,val)

dataset = df['my_dataset']
data = dataset[:]

print((data))
# print((data)[0])
doc_map = {}
# print(len(data))
for i in range(len(data)):
    # print(data)
    doc_id = data[i][0]
    doc_location = data[i][1].decode('UTF-8')
    
    doc_map[doc_id] = doc_location

# for doc_id, doc_location in doc_map.items():
#     print(doc_id,doc_location)
print(doc_map)
with open('generated.json', 'r') as f:
    data_json = json.load(f)

# # print(data_json)

[(0, b'640c08980f') (1, b'640c0898a3')]
{0: '640c08980f', 1: '640c0898a3'}


In [3]:
for keyword_obj in data_json:
    key_arr = keyword_obj['keywords_array']
    
    for keyarr_obj in range(len(key_arr)):

        key_obj = key_arr[keyarr_obj]
        # print(key_obj)
        keyword_name = key_obj['keyword']
        keyword_score = key_obj['score']
        
        # print(keyword_name,keyword_score)

        file2 = h5py.File(keyword_name + "Index.hdf5", 'a')
        file3 = h5py.File(keyword_name + "Bias.hdf5", 'a')
        
        if keyword_name not in file2:
            dataset2 = file2.create_dataset(keyword_name, shape=(0,), maxshape=(None,), dtype = [('id', int), ('score', float)])
        else:
            dataset2 = file2[keyword_name]
        
        if keyword_name not in file3:
            dataset3 = file3.create_dataset(keyword_name, shape=(0,), maxshape=(None,), dtype = [('id', int),('bias', int)])
        else:
            dataset3 = file3[keyword_name]

        for doc_id, doc_location in doc_map.items():

            if doc_location in keyword_obj['doc_loc']:
                
                dataset2.resize((dataset2.shape[0] + 1,))
                dataset3.resize((dataset3.shape[0] + 1,))
                # print(doc_location,keyword_score)
                # print(type(int(doc_location)),type(keyword_score))
                dataset2[-1] = (doc_id, keyword_score)
                dataset3[-1:] = (doc_id, 1)